In [1]:
import torch

In [2]:
num_entries = 5000
db = torch.rand(num_entries) > 0.5
db

tensor([1, 0, 1,  ..., 1, 1, 1], dtype=torch.uint8)

In [3]:
db[0:5]

tensor([1, 0, 1, 1, 1], dtype=torch.uint8)

In [4]:
remove_index = 2
torch.cat((db[0:2], db[3:]))[0:5]

tensor([1, 0, 1, 1, 1], dtype=torch.uint8)

In [5]:
def get_parallel_db(db, remove_index):
    return torch.cat((db[0:remove_index], 
                      db[remove_index+1:]))

In [6]:
parallel_dbs = list()

In [7]:
def get_parallel_dbs(db):
    parallel_dbs = list()
    for i in range(len(db)):
        pdb = get_parallel_db(db, i)
        parallel_dbs.append(pdb)
    return parallel_dbs

In [8]:
pdbs = get_parallel_dbs(db)

In [9]:
def create_db_and_parallels(num_entries):
    db = torch.rand(num_entries) > 0.5
    pdbs = get_parallel_dbs(db)
    return db, pdbs

In [10]:
db, pdbs = create_db_and_parallels(20)

In [48]:
def query(db):
    return db.sum()

In [49]:
def sensitivity(query, n_entries):
    db, pdbs = create_db_and_parallels(5000)
    full_db_result = query(db)
    max_distance = 0
    for pdb in pdbs:
        pdb_result = query(pdb)
        db_distance = torch.abs(pdb_result - full_db_result)    
        if db_distance > max_distance:
            max_distance = db_distance
    return max_distance

In [50]:
def query(db):
    return db.float().mean()

In [51]:
sensitivity(query, 1000)

tensor(0.0001)

In [52]:
0.1/1000

0.0001

In [53]:
#Calculate L1 sensitivity for threshold

In [54]:
def query(db, threshold=5):
    return (db.sum() > threshold)

In [55]:
for i in range(10):
    sens_f = sensitivity(query, n_entries=10)
    #print(query)
    print(sens_f)

0
0
0
0
0
0
0
0
0
0


In [56]:
#Perform differencing attack on row 10

In [57]:
db,_ = create_db_and_parallels(100)

In [58]:
db

tensor([0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0,
        1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1,
        1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1,
        0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0,
        0, 0, 0, 1], dtype=torch.uint8)

In [59]:
pdb = get_parallel_db(db, remove_index=10)

In [23]:
db[10]

tensor(1, dtype=torch.uint8)

In [24]:
sum(db) - sum(pdb)

tensor(1, dtype=torch.uint8)

In [25]:
sum(db)

tensor(50, dtype=torch.uint8)

In [26]:
#differencing attack using sum query
sum(db) - sum(pdb)

tensor(1, dtype=torch.uint8)

In [27]:
#differencing attack using mean query
(sum(db).float() / len(db)) - (sum(pdb).float() / len(pdb))

tensor(0.0051)

In [28]:
#differencing attack using mean query
(sum(db).float() > 49) - (sum(pdb).float() > 49)

tensor(1, dtype=torch.uint8)

In [29]:
#Implement local differential privacy

In [30]:
db, pdbs = create_db_and_parallels(100)

In [31]:
true_result = torch.mean(db.float())
true_result

tensor(0.5000)

db

In [32]:
first_coin_flip = (torch.rand(len(db)) > 0.5).float()
second_coin_flip = (torch.rand(len(db)) > 0.5).float()
first_coin_flip

tensor([1., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1.,
        0., 0., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 0., 1., 1.,
        1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 0., 0., 1., 1., 1., 0., 0., 1.,
        0., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.,
        1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0.,
        0., 1., 0., 0., 0., 0., 1., 1., 0., 0.])

In [33]:
second_coin_flip

tensor([0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
        1., 0., 0., 1., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0., 1., 1., 0., 0.,
        0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1.,
        0., 0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 1., 0.,
        0., 0., 1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 0.,
        1., 1., 1., 1., 1., 1., 0., 0., 1., 0.])

###Now The first coin flip will determine whether we will use the original value in the database or will use the value that is randonly generated in the second coin flip. We can do this by simply multiplying the first coin flip to the db. because If the first coin flip value is 1 that means the corrsponding answer in the db is true. If it is zero, we don't take the corresponding db value. 

But sometimes people will lie.We need to go for second coin flip. 

In [34]:
db.float() * first_coin_flip

tensor([0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0.,
        0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1.,
        1., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 1.,
        0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0.,
        0., 1., 0., 0., 0., 0., 1., 0., 0., 0.])

In [35]:
(1-first_coin_flip)

tensor([0., 0., 1., 0., 0., 1., 1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0.,
        1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0.,
        0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 1., 0.,
        1., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.,
        0., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1.,
        1., 0., 1., 1., 1., 1., 0., 0., 1., 1.])

###Sometimes we need to choose randomly. in the abobe mention result where there is a 1 that means we need to choose randomly through a second coin flip.

In [36]:
(1-first_coin_flip) * second_coin_flip

tensor([0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0.,
        1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 1., 0., 0.,
        0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0.,
        1., 0., 1., 1., 1., 1., 0., 0., 1., 0.])

If we just add them together, we will get a new augmented database that is differentially private

In [37]:
augmented_databse = (db.float() * first_coin_flip) + ((1-first_coin_flip) * second_coin_flip)

In [38]:
augmented_databse

tensor([0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 0., 1., 1., 1., 0., 0.,
        1., 0., 1., 1., 1., 0., 0., 1., 1., 0., 1., 1., 0., 0., 1., 1., 0., 1.,
        1., 1., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1.,
        0., 1., 1., 1., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 1., 1., 0., 0.,
        0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0.,
        1., 1., 1., 1., 1., 1., 1., 0., 1., 0.])

In [39]:
torch.mean(augmented_databse.float())

tensor(0.5600)

We know that half the time the result was honest, and rest half of the time it was random. That in that rest hlf time there is 50/50 chance that result is honest. That gives around 70% honest answer. The taboo behaviour was 70% too. But because of that rest 50% random result, our result will always try to scew towards 50%. Now let's try to deskew the results

In [40]:
torch.mean(augmented_databse.float()) * 2 - 0.5

tensor(0.6200)

So This is the original result. Let's make a function

In [41]:
def query(db):
    true_result = torch.mean(db.float())
    first_coin_flip = (torch.rand(len(db)) > 0.5).float()
    second_coin_flip = (torch.rand(len(db)) > 0.5).float()
    augmented_databse = db.float() * first_coin_flip + (1-first_coin_flip) * second_coin_flip
    dp_result = torch.mean(augmented_databse.float()) * 2 - 0.5
    return dp_result, true_result  

In [42]:
query(db)

(tensor(0.6000), tensor(0.5000))

In [43]:
db, pdbs = create_db_and_parallels(10)
private_result, true_result = query(db)
print("With Noise:" + str(private_result))
print("Without Noise:" + str(true_result))

With Noise:tensor(0.1000)
Without Noise:tensor(0.4000)


In [44]:
db, pdbs = create_db_and_parallels(100)
private_result, true_result = query(db)
print("With Noise:" + str(private_result))
print("Without Noise:" + str(true_result))

With Noise:tensor(0.4600)
Without Noise:tensor(0.5100)


In [45]:
db, pdbs = create_db_and_parallels(1000)
private_result, true_result = query(db)
print("With Noise:" + str(private_result))
print("Without Noise:" + str(true_result))

With Noise:tensor(0.4960)
Without Noise:tensor(0.5270)


In [46]:
db, pdbs = create_db_and_parallels(10000)
private_result, true_result = query(db)
print("With Noise:" + str(private_result))
print("Without Noise:" + str(true_result))

With Noise:tensor(0.5106)
Without Noise:tensor(0.5062)


If you notice, in this results, bigger the database smaller the effect os noise. When we are adding noise, we are corrupting the dataset. It definitely has an impact on the query. However, the more datapoints we have, the noise effect is tend to average out by the number of data. Local differential privacy is more data hungry. Because we add noise in every data point. To average it out it requires more data. On the other hand global differential privacy is less data hungry, because we add noise only in the output. So the query is less corrupted and more accurate. When the database is not too big, it's a better idea to use global differential privacy.

The advantage of the local DP is we don't have to tust someone with our data and also we can publish our individual data. 